<a href="https://colab.research.google.com/github/AtulAravindDas/DeepLearningAI_CrewAI_Agents/blob/main/Automated_Event_Planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prerequisite notes**

*   Tasks: Cornerstone of multiagent systems

*   Consider the manager analogy: Make the LLM think like a manager and do tasks like a manager like who is the right fit for the job and stuff

*   Have a clear description and explain it truthfully

*   Set a clear and concise expectation

*   This ensures the building of better internal prompts

*   Set context

*   Set callback

*   Override agent with specific task tools

*   Output as JSON












# **Initializing all libraries**

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
from crewai import Agent,Task,Crew

In [3]:
import warnings
warnings.filterwarnings('ignore')

# **Setting up the environment variables**

In [4]:
import os
import getpass

api_key=os.environ.get("OPENAI_API_KEY")

if api_key:
  openai_api_key=api_key
  os.environ["OPENAI_MODEL_NAME"]='gpt-4o'
  print("OpenAI API Key already set")
else:
  api_key=getpass.getpass("OpenAI API Key: ")
  openai_api_key=api_key
  serper_api_key=getpass.getpass("SerperDev API Key: ")
  os.environ["SERPER_API_KEY"]=serper_api_key
  os.environ["OPENAI_API_KEY"]=openai_api_key
  os.environ["OPENAI_MODEL_NAME"]='gpt-4o'
  print("OpenAI API Key set")
  print("SerperDev API Key Set")

OpenAI API Key: ··········
SerperDev API Key: ··········
OpenAI API Key set
SerperDev API Key Set


# **Setting the tools to do the work**

* SerperDevTool: Search google for a particular text or input and get resulting websites
* ScrapeWebsiteTool: Scrape the resulting websites and access their contents

In [5]:
from crewai_tools import SerperDevTool,ScrapeWebsiteTool

search_tools=SerperDevTool()
scrape_tools=ScrapeWebsiteTool()

# Agent Creation

The following agents will be required for the job

| Agent |  Goal |
|-------|-------|
|Venue Coordinator |Identify and book an appropriate venue based on event requirements|
|Logistics Manageer | Manage Logistics for the Event including catering and equipment |
| Marketing Agent | How do we market this event/meetup |



In [7]:
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tools, scrape_tools],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [8]:
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tools, scrape_tools],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [10]:
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tools, scrape_tools],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

# Create a venue Pydantic Object

* Pydantic: Create a Model in work easily with typed up variables
*  Create a class VenueDetails using Pydantic BaseModel.
* Agents will populate this object with information about different venues by creating different instances of it as they work.
* It results in strong typed outputs

**TL:DR: Pydantic Baemodel class used to store venue info**


In [11]:
from pydantic import BaseModel
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

# **Create Tasks**

The following tasks are required to be done

| Task | Description |
| ----- | ---------- |
| venue_task | Finding the ideal event location in an event city for an event topic and event duration. |
| logistic_task | Coordinate with teams such as catering and equipment |
| marketing_task | Promote this event by doing all the marketing activities |


In [12]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [15]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True, #Before task is completed, the agents will ask us if we are satisfied with the output
    async_execution=True, #Executed in parallel with any tasks after this
    agent=logistics_manager
)

In [16]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

# **Creating the crew**

In [17]:
event_management_crew=Crew(agents=[venue_coordinator,logistics_manager,marketing_communications_agent],tasks=[venue_task,logistics_task,marketing_task],verbose=True)

# **Running the crew**

In [18]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "Boston",
    'tentative_date': "2025-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall in Boston University"
}

In [22]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in Boston that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a suitable venue in Boston for the Tech Innovation Conference. To do this, I should start by understanding the specific requirements for the event, such as the expected number of attendees, budget constraints, and any special needs or themes. Since I don't have these details readily available, I'll need to ask for them.

Action: Ask question to co-worker
Action Input: {
    "coworker": "Logistics Manager",
    "question": "What are the specific requirements for the Tech Innovation Conference in terms of expected number of attendees, budget constraints, and any special needs or themes for the venue?",
    "context": "I need to find a suitable venue in Boston for the Tech Innovation Conference. To do this effectively, I require detailed information about the event's size, budget, and 

In [23]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '33rd floor, 60 State St, Boston',
 'booking_status': 'Booked for Tech Innovation Summit 2025 on June 10, 2025',
 'capacity': 200,
 'name': 'The State Room'}


In [24]:
from IPython.display import Markdown
Markdown("marketing_report.md")

```markdown
# Marketing Plan for Tech Innovation Conference

## Event Details
- **Name:** Tech Innovation Conference
- **Location:** Boston Convention and Exhibition Center (BCEC), located just 2.9 miles from Logan International Airport
- **Capacity:** 516,000
- **Booking Status:** Contact the sales team via email or at 877-393-3393 for booking and inquiries.

## Marketing Strategies

### 1. Identify and Segment Target Audience
- Conduct market research to understand the demographics, interests, and behaviors of potential attendees.
- Create audience personas to tailor content and messaging.
- Segment the audience based on demographics, psychographics, and technology use to personalize engagement.

### 2. Optimize Event Website
- Develop a user-friendly, mobile-responsive event website.
- Ensure the website includes clear calls-to-action, easy navigation, and detailed event information.
- Use consistent branding and design elements to enhance user experience.

### 3. Email Marketing Campaigns
- Design personalized email campaigns to engage different audience segments.
- Use recipient's names and tailor content based on their preferences and behaviors.
- Include strong call-to-actions to drive registrations.

### 4. Social Media Engagement
- Launch social media campaigns using platforms like LinkedIn, Twitter, and Facebook.
- Create a unique event hashtag to facilitate conversations.
- Share regular updates, behind-the-scenes content, and engaging visuals.
- Host giveaways and contests to boost participation.

### 5. Leverage Speakers and Influencers
- Encourage guest speakers to promote the event through their networks.
- Provide speakers with marketing collateral, including shareable links and event details.
- Partner with industry influencers to expand reach and credibility.

### 6. Video Marketing
- Create promotional videos showcasing event highlights, speaker insights, and venue tours.
- Share video testimonials from past attendees to build trust and excitement.
- Use live streaming to engage remote audiences and increase FOMO.

### 7. Offer Incentives
- Provide exclusive content, discounts, or perks as incentives for early registrations.
- Use raffles or sweepstakes to encourage participation and data collection.

### 8. Press Releases and Media Outreach
- Distribute press releases to industry publications and media outlets.
- Highlight the conference's unique selling points and value propositions.

## Conclusion
By implementing these marketing strategies, the Tech Innovation Conference aims to engage and attract at least 500 potential attendees. These efforts will increase brand awareness, foster community engagement, and establish the conference as a leading event in the tech industry.
```